## Facial Emotion Recognition using AlexNet

### Import and Installation

In [15]:
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import numpy as np
import pathlib
import datetime



# printout versions
%pip install tensorflow-addons
print(f"Tensorflow version: {tf.__version__}")
print(f"Numpy version: {np.version.version}")

Note: you may need to restart the kernel to use updated packages.
Tensorflow version: 2.12.0
Numpy version: 1.23.5


### Get/Set Data Info

In [16]:
data_train_dir = pathlib.Path("./dataset/train/")
data_test_dir = pathlib.Path("./dataset/test")
image_count = len(list(data_train_dir.glob('*/*.png')))
BATCH_SIZE = 32             
IMG_HEIGHT = 227            
IMG_WIDTH = 227             
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
print(image_count)

class_names = np.array([item.name for item in data_train_dir.glob('*') if item.name != ".DS_Store"])
print("Class Names:\n" + str(class_names))
class_count = len(class_names)
print("Class Count:\n" + str(class_count))

28709
Class Names:
['happy' 'sad' 'fearful' 'neutral' 'angry' 'disgusted' 'surprised']
Class Count:
7


### AlexaNet Model

In [17]:
model = tf.keras.models.Sequential([
  # Conv Layer 1
  tf.keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=(227, 227, 3)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
  # Conv Layer 2
  tf.keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  # Conv Layer 3
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  # Conv Layer 4
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  # Conv Layer 5
  tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # Flattern after Conv layers
  tf.keras.layers.Flatten(),
  # FC1
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  # FC2
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  # FC3 classifier  
  tf.keras.layers.Dense(class_count, activation='sigmoid')
])

### Data Generator 

In [18]:
# normalize images to [0-1] interval
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# training_data for model training
train_data_generator = image_generator.flow_from_directory(directory=str(data_train_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH), 
                                                     classes = list(class_names))

test_data_generator = image_generator.flow_from_directory(directory=str(data_test_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH), 
                                                     classes = list(class_names))

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


### Model

In [19]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.SensitivityAtSpecificity(0.5), tf.keras.metrics.SpecificityAtSensitivity(0.5), tf.keras.metrics.AUC(curve='ROC')])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization_25 (Bat  (None, 55, 55, 96)       384       
 chNormalization)                                                
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 27, 27, 256)       2973952   
                                                                 
 batch_normalization_26 (Bat  (None, 27, 27, 256)      1024      
 chNormalization)                                                
                                                      

### Train

In [20]:
import time
start = time.time()
history = model.fit(train_data_generator, steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=10, validation_data=test_data_generator)

# Saving the model
model.save('AlexNet_FER_Model/')
print("Total time: ", time.time() - start, "seconds")

Epoch 1/10


2023-04-13 04:25:47.963812: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 49/898 [>.............................] - ETA: 4:44:16 - loss: 18.2355 - accuracy: 0.1888 - precision: 0.1652 - recall: 0.6722 - sensitivity_at_specificity: 0.6046 - specificity_at_sensitivity: 0.5890 - auc: 0.5668

KeyboardInterrupt: 

### Plot result

In [ ]:
def plot_result(history):
    plt.plot(history.history["val_accuracy"],color='green')
    plt.plot(history.history["accuracy"],color='olive')
    plt.plot(history.history["precision"],color='violet')
    plt.plot(history.history["val_precision"],color='purple')
    plt.plot(history.history["recall"],color='cyan')
    plt.plot(history.history["val_recall"],color='yellow')
    plt.title("Metrics")
    
    plt.legend(["Test Accuracy", "Train Accuracy", "Train Precision", "Test Precision", "Train Recall", "Test Recall" ], bbox_to_anchor =(0.65, 1.00))
    plt.show()

plot_result(history)